**About** : This notebook is used to validate models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
torch.cuda.get_device_name(0)

In [ ]:
import os
import re
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
pd.set_option('display.max_columns', 500)

In [ ]:
from params import *

from data.preparation import *
from data.dataset import SignDataset

from utils.metrics import *
from utils.plots import *
from utils.logger import Config

from inference.main import kfold_inference_val

### Expes
- Make sure you've generated the predictions using the inference part of the notebook

In [ ]:
df = prepare_data(DATA_PATH, "")

In [ ]:
FILES = [   # CV 0.7490 - LB 0.79  576-80_768-25_norm-soup
    "../logs/2023-04-30/7/pred_oof_dist_soup.npy",
    "../logs/2023-05-01/2/pred_oof_dist_soup.npy",
]

pred_oof = np.mean([np.load(f) for f in FILES], 0)

df['pred'] = pred_oof.argmax(-1)

score = accuracy(df['target'], pred_oof)
print(f"\n-> CV acc : {score:.4f}")

### Inference

In [ ]:
EXP_FOLDER = "../logs/2023-04-30/7/"  # 0.7380 / DIST 0.7395 / DISTx10 0.7402    torch_12 768 mix norm
EXP_FOLDER = "../logs/2023-05-01/2/"  # 0.7395 / DIST 0.7394 / DISTx10 0.7395    torch_18 576 mix norm

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))
df = prepare_data(DATA_PATH, config.processed_folder)

if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])
    
try:
    pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")
    df['pred'] = pred_oof.argmax(-1)

    score = accuracy(df['target'], pred_oof)
    print(f"-> CV acc : {score:.4f}")
except:
    df = df[df['fold'] == 0].reset_index(drop=True)
    
    pred_oof = np.load(EXP_FOLDER + "pred_val_0.npy")
    df['pred'] = pred_oof.argmax(-1)

    score = accuracy(df['target'], pred_oof)
    print(f"-> Fold 0 acc : {score:.4f}")

In [ ]:
pred_oof = kfold_inference_val(
    df,
    EXP_FOLDER,
    debug=False,
    save=False,
    use_fp16=True,
    use_mt=False,
    distilled=True,
    n_soup=10,
    train=False,
)

Done ! 